In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

In [2]:
raw_data = pd.read_csv("/Users/hnwang/ihm/without_fillna/alldata.csv")
med_columns = raw_data.columns[raw_data.columns.str.contains('Med_')]

# fill Med NaN to 0
raw_data[med_columns] = raw_data[med_columns].fillna(0)

In [3]:
columns_to_rename = raw_data.columns[65:273]

# rename
new_column_names = [col.rsplit('_', 1)[0] for col in columns_to_rename]

raw_data.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)
filtered_df = raw_data[raw_data['Is transferred to ICU'] == 1].drop('Is transferred to ICU', axis=1, errors='ignore')

In [4]:
filtered_df.columns[:20]

Index(['stay_ID', 'patient_ID', 'Age', 'Gender', 'Insurance', 'Race',
       'Arrival transport', 'triage_Temperature', 'triage_HeartRate',
       'triage_RespiratoryRate', 'triage_SpO2', 'triage_SBP', 'triage_DBP',
       'triage_Acuity', 'vitalsigns_Temperature', 'vitalsigns_HeartRate',
       'vitalsigns_RespiratoryRate', 'vitalsigns_SpO2', 'vitalsigns_SBP',
       'vitalsigns_DBP'],
      dtype='object')

In [5]:
filtered_df.dtypes[-10:]

Diagnosis_S06                          float64
Diagnosis_S72                          float64
Diagnosis_W18                          float64
Diagnosis_W19                          float64
Diagnosis_X58                          float64
Diagnosis_Y83                          float64
Diagnosis_Z79                          float64
Emergency department length of stay    float64
Emergency department boarding time     float64
IHM                                      int64
dtype: object

In [6]:
transform_dtypes_data = filtered_df.copy().iloc[:,2:]

category_columns_1 = transform_dtypes_data.columns[transform_dtypes_data.columns.str.contains('Med_') | transform_dtypes_data.columns.str.startswith('Diagnosis_')]
transform_dtypes_data[category_columns_1] = transform_dtypes_data[category_columns_1].applymap(lambda x: 0 if x == 0.0 else 1)
transform_dtypes_data[category_columns_1] = transform_dtypes_data[category_columns_1].astype('category')

category_columns_2 = ['Gender', 'Insurance', 'Race', 'Arrival transport','triage_Acuity', 'IHM']
transform_dtypes_data[category_columns_2] = transform_dtypes_data[category_columns_2].astype('category')

print(transform_dtypes_data.dtypes)

Age                                       int64
Gender                                 category
Insurance                              category
Race                                   category
Arrival transport                      category
                                         ...   
Diagnosis_Y83                          category
Diagnosis_Z79                          category
Emergency department length of stay     float64
Emergency department boarding time      float64
IHM                                    category
Length: 274, dtype: object


In [7]:
dtypes_dict = {}

for dtype in transform_dtypes_data.dtypes.unique():
    dtypes_dict[str(dtype)] = transform_dtypes_data.select_dtypes(include=[dtype]).columns.tolist()

for dtype, columns in dtypes_dict.items():
    print(f"Data type: {dtype}")
    print(f"Number of columns: {len(columns)}")
    print(f"Columns: {columns}\n")

Data type: int64
Number of columns: 1
Columns: ['Age']

Data type: category
Number of columns: 214
Columns: ['Gender', 'Insurance', 'Race', 'Arrival transport', 'triage_Acuity', 'Med_2747.0', 'Med_445.0', 'Med_575.0', 'Med_5843.0', 'Med_2530.0', 'Med_670.0', 'Med_583.0', 'Med_250.0', 'Med_6615.0', 'Med_224.0', 'Med_523.0', 'Med_530.0', 'Med_577.0', 'Med_414.0', 'Med_701.0', 'Med_5970.0', 'Med_4609.0', 'Med_6030.0', 'Med_6081.0', 'Med_411.0', 'Med_2750.0', 'Med_6094.0', 'Med_156.0', 'Med_806.0', 'Med_409.0', 'Med_225.0', 'Med_785.0', 'Med_443.0', 'Med_206.0', 'Med_457.0', 'Med_5904.0', 'Med_254.0', 'Med_6089.0', 'Med_781.0', 'Med_301.0', 'Med_6143.0', 'Med_784.0', 'Med_2812.0', 'Med_290.0', 'Med_3179.0', 'Med_6126.0', 'Med_214.0', 'Med_3326.0', 'Med_521.0', 'Med_3925.0', 'Med_5795.0', 'Med_3510.0', 'Med_2602.0', 'Med_2683.0', 'Med_6043.0', 'Med_2587.0', 'Med_5658.0', 'Med_2578.0', 'Med_649.0', 'Med_371.0', 'Med_733.0', 'Med_5733.0', 'Med_3948.0', 'Med_4544.0', 'Med_704.0', 'Med_3932.0',

In [8]:
from tableone import TableOne
df = transform_dtypes_data.copy()

In [9]:

features = [
    'Age', 'Gender', 'Race', 'Arrival transport', 'Insurance',
    'triage_HeartRate', 'triage_RespiratoryRate',  'triage_SpO2',
    'vitalsigns_Temperature', 'vitalsigns_HeartRate','vitalsigns_SBP', 'vitalsigns_DBP',
    'Lab_Albumin', 'Lab_Lymphocytes', 'Lab_Bicarbonate', 'Lab_Bilirubin', 'Lab_Chloride', 'Lab_Creatinine', 'Lab_Glucose', 'Lab_Hematocrit', 'Lab_Platelet Count', 'Lab_Potassium', 'Lab_White Blood Cells',
    'Med_2747.0','Med_583.0', 'Med_250.0', 'Med_1247.0',
    'Diagnosis_J18', 'Diagnosis_R06',
    'triage_Acuity',
    # 'heartrate_last', 'sysbp_last', 'diasbp_last', 'tempc_last', 'spo2_last',
    # 'albumin_last', 'glucose_last', 'wbc_last', 'bicarbonate_last', 'potassium_last', 'platelet_last', 'bilirubin_last','hematocrit_last', 'chloride_last'
]

category_columns = ['Gender', 'Race', 'Arrival transport','Insurance', 'Med_2747.0', 'Med_583.0', 'Med_250.0', 'Med_1247.0','triage_Acuity', 'Diagnosis_J18', 'Diagnosis_R06']

nonnormal = [
    'Age',
     'triage_RespiratoryRate','triage_HeartRate', 'triage_SpO2',
    'vitalsigns_Temperature', 'vitalsigns_HeartRate', 'vitalsigns_SBP', 'vitalsigns_DBP',
    'Lab_Albumin', 'Lab_Lymphocytes', 'Lab_Bicarbonate', 'Lab_Bilirubin', 'Lab_Chloride', 'Lab_Creatinine', 'Lab_Glucose', 'Lab_Hematocrit', 'Lab_Platelet Count', 'Lab_Potassium', 'Lab_White Blood Cells',
]

table = TableOne(
    data=df,
    columns=features,
    categorical=category_columns,
    groupby='IHM',
    pval=True,
    nonnormal=nonnormal,
    decimals=1, htest_name=False

)

print(table)

                                           Grouped by IHM                                                                       
                                                  Missing              Overall                    0                    1 P-Value
n                                                                        29197                25978                 3219        
Age, median [Q1,Q3]                                     0     67.0 [54.0,79.0]     66.0 [54.0,78.0]     74.0 [62.0,84.0]  <0.001
Gender, n (%)                          0                          13532 (46.3)         12028 (46.3)          1504 (46.7)   0.664
                                       1                          15665 (53.7)         13950 (53.7)          1715 (53.3)        
Race, n (%)                            0                            2049 (7.0)           1671 (6.4)           378 (11.7)  <0.001
                                       1                          19347 (66.3)         17247 (66.

## Step 1: Univariate Feature Selection  
In this step, each feature is independently evaluated for its correlation with the target variable. For categorical features, tests like the Chi-square test are used, while for continuous features, methods such as the t-test or Wilcoxon rank-sum test are applied. Features with statistically significant correlations (e.g., p < 0.05) are selected, which helps remove redundant features and retain those most relevant to the target.

In [10]:
transform_dtypes_data.describe()

,Age,triage_Temperature,triage_HeartRate,triage_RespiratoryRate,triage_SpO2,triage_SBP,triage_DBP,vitalsigns_Temperature,vitalsigns_HeartRate,vitalsigns_RespiratoryRate,...,Lab_RDW,Lab_Red Blood Cells,Lab_Sodium,Lab_Urea Nitrogen,Lab_White Blood Cells,Lab_RBC,Lab_Specific Gravity,Lab_pH,Emergency department length of stay,Emergency department boarding time
count,29197.000000,22541.000000,24511.000000,22198.000000,23510.000000,24477.000000,24345.000000,26071.000000,27959.000000,27862.000000,...,27684.000000,27695.000000,26461.000000,27715.000000,27714.000000,11677.000000,15473.000000,16387.000000,29197.000000,29197.000000
mean,65.201562,98.117588,91.206091,18.004505,97.333003,129.285100,72.928240,98.257669,87.119622,19.062199,...,15.142505,3.901045,137.191829,29.088147,11.841888,10.290486,1.018390,6.232227,346.555384,71.147636
std,17.556876,1.014572,22.303735,2.417775,2.477151,28.151516,17.416372,0.780968,19.948020,3.993018,...,2.525025,0.873870,6.084130,24.429036,11.253058,23.524285,0.009522,0.712160,217.620363,60.104707
min,18.000000,94.600000,14.000000,10.000000,88.000000,40.000000,4.000000,95.400000,1.000000,4.000000,...,0.000000,0.000000,90.000000,1.000000,0.000000,0.000000,1.000000,5.000000,8.183333,0.350000
25%,54.000000,97.500000,75.000000,16.000000,96.000000,109.000000,61.000000,97.800000,73.000000,16.000000,...,13.300000,3.320000,134.000000,14.000000,7.200000,1.000000,1.012000,6.000000,202.000000,2.966667
50%,67.000000,98.000000,89.000000,18.000000,98.000000,127.000000,72.000000,98.100000,86.000000,18.000000,...,14.500000,3.960000,138.000000,21.000000,10.000000,2.000000,1.017000,6.000000,298.000000,70.000000
75%,79.000000,98.600000,106.000000,20.000000,99.000000,147.000000,84.000000,98.600000,100.000000,21.000000,...,16.300000,4.510000,141.000000,35.000000,14.000000,7.000000,1.023000,6.500000,430.000000,103.000000
max,102.000000,101.600000,177.000000,24.000000,100.000000,245.000000,147.000000,100.900000,169.000000,32.000000,...,34.100000,7.750000,180.000000,260.000000,402.500000,182.000000,1.052000,9.000000,1854.000000,605.000000


In [11]:
def remove_outliers(df):
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            lower_bound = df[column].quantile(0.01)
            upper_bound = df[column].quantile(0.99)
            df[column] = df[column].apply(lambda v: lower_bound if v < lower_bound else upper_bound if v > upper_bound else v)
        if column == 'Age':
            df[column] = df[column].apply(lambda v: 91 if v > 91 else v)
        return df


In [12]:
cleaned_data = remove_outliers(transform_dtypes_data)

In [13]:
cleaned_data.describe()

,Age,triage_Temperature,triage_HeartRate,triage_RespiratoryRate,triage_SpO2,triage_SBP,triage_DBP,vitalsigns_Temperature,vitalsigns_HeartRate,vitalsigns_RespiratoryRate,...,Lab_RDW,Lab_Red Blood Cells,Lab_Sodium,Lab_Urea Nitrogen,Lab_White Blood Cells,Lab_RBC,Lab_Specific Gravity,Lab_pH,Emergency department length of stay,Emergency department boarding time
count,29197.000000,22541.000000,24511.000000,22198.000000,23510.000000,24477.000000,24345.000000,26071.000000,27959.000000,27862.000000,...,27684.000000,27695.000000,26461.000000,27715.000000,27714.000000,11677.000000,15473.000000,16387.000000,29197.000000,29197.000000
mean,65.143063,98.117588,91.206091,18.004505,97.333003,129.285100,72.928240,98.257669,87.119622,19.062199,...,15.142505,3.901045,137.191829,29.088147,11.841888,10.290486,1.018390,6.232227,346.555384,71.147636
std,17.409553,1.014572,22.303735,2.417775,2.477151,28.151516,17.416372,0.780968,19.948020,3.993018,...,2.525025,0.873870,6.084130,24.429036,11.253058,23.524285,0.009522,0.712160,217.620363,60.104707
min,21.000000,94.600000,14.000000,10.000000,88.000000,40.000000,4.000000,95.400000,1.000000,4.000000,...,0.000000,0.000000,90.000000,1.000000,0.000000,0.000000,1.000000,5.000000,8.183333,0.350000
25%,54.000000,97.500000,75.000000,16.000000,96.000000,109.000000,61.000000,97.800000,73.000000,16.000000,...,13.300000,3.320000,134.000000,14.000000,7.200000,1.000000,1.012000,6.000000,202.000000,2.966667
50%,67.000000,98.000000,89.000000,18.000000,98.000000,127.000000,72.000000,98.100000,86.000000,18.000000,...,14.500000,3.960000,138.000000,21.000000,10.000000,2.000000,1.017000,6.000000,298.000000,70.000000
75%,79.000000,98.600000,106.000000,20.000000,99.000000,147.000000,84.000000,98.600000,100.000000,21.000000,...,16.300000,4.510000,141.000000,35.000000,14.000000,7.000000,1.023000,6.500000,430.000000,103.000000
max,91.000000,101.600000,177.000000,24.000000,100.000000,245.000000,147.000000,100.900000,169.000000,32.000000,...,34.100000,7.750000,180.000000,260.000000,402.500000,182.000000,1.052000,9.000000,1854.000000,605.000000


In [14]:
import pandas as pd
from scipy import stats
import numpy as np

def remove_outliers(x):
    lower_bound = x.quantile(0.01)
    upper_bound = x.quantile(0.99)
    # return x[(x >= lower_bound) & (x <= upper_bound)]
    x = x.apply(lambda v: lower_bound if v < lower_bound else upper_bound if v > upper_bound else v)
    return x
    
def perform_test(x, y):
    if pd.api.types.is_numeric_dtype(x):
        # x.fillna(x.median(), inplace=True)
        group1 = remove_outliers(x[y == 1].dropna())
        group2 = remove_outliers(x[y != 1].dropna())
        
        if stats.shapiro(x.dropna())[1] > 0.05:
            # print('number ttest_ind')
            stat, pvalue = stats.ttest_ind(group1, group2)
        else:
            if len(group1) > 1 and len(group2) > 1 and not group1.equals(group2):
                # print('number mannwhitneyu')
                stat, pvalue = stats.mannwhitneyu(group1, group2)
                print(pvalue, '*'*10)
            else:
                stat, pvalue = np.nan, np.nan
    else:
        # print('cate chi2')
        crosstab = pd.crosstab(x, y)
        stat, pvalue, _, _ = stats.chi2_contingency(crosstab)

        print(pvalue)

    if pvalue is not None:
        pvalue = np.round(pvalue, 4)
    h = 1 if pvalue < 0.05 else 0
    return pvalue, h

def all_tests(X, y):
    results = []
    for i in X.columns:  
        print(f"Processing {i}")
        pvalue, h = perform_test(X[i], y)
        results.append([i, pvalue, h])
    
    results_df = pd.DataFrame(results, columns=['ID', 'pvalue', 'h'])
    return results_df

def get_significant_features(X, y, save_path):
    # X.fillna(X.median(), inplace=True)  # 
    all_results = all_tests(X, y)
    all_results.to_excel(save_path + 'rawData_allTestsPvalue_ihm.xlsx', index=False)
    
    significant = all_results[all_results['h'] == 1]
    significant.to_excel(save_path + 'significantPvalue_ihm.xlsx', index=False)
    
    significant_ids = significant['ID']
    significant_data = X.loc[:, significant_ids]
    final_data = pd.concat([significant_data, y], axis=1)
    final_data.reset_index(drop=True, inplace=True)
    final_data.to_csv(save_path + 'data_FScorr_step1_ihm.csv', index=False)
    return final_data

save_path = '/Users/hnwang/bmc/'
final_data = get_significant_features(transform_dtypes_data.iloc[:,:-1], transform_dtypes_data.iloc[:,-1], save_path)

Processing Age
2.90954921873885e-132 **********
Processing Gender
0.6642741501484717
Processing Insurance
1.5250491112999056e-14
Processing Race
7.922924703129161e-31
Processing Arrival transport
3.349444370120337e-28
Processing triage_Temperature
1.511574756094908e-09 **********
Processing triage_HeartRate
0.0002852464767398263 **********
Processing triage_RespiratoryRate
4.244173906760515e-16 **********
Processing triage_SpO2
4.641546900057491e-17 **********
Processing triage_SBP
8.916751874777989e-57 **********
Processing triage_DBP
7.317051396630579e-44 **********
Processing triage_Acuity
7.768037655824192e-42
Processing vitalsigns_Temperature
1.2300623531705648e-22 **********
Processing vitalsigns_HeartRate
7.487336313450308e-25 **********
Processing vitalsigns_RespiratoryRate
2.1190127469141106e-37 **********
Processing vitalsigns_SpO2
0.15898029414217973 **********
Processing vitalsigns_SBP
6.384235532080234e-47 **********
Processing vitalsigns_DBP
1.9145333883647935e-32 *******

In [15]:
final_data.describe()

,Age,triage_Temperature,triage_HeartRate,triage_RespiratoryRate,triage_SpO2,triage_SBP,triage_DBP,vitalsigns_Temperature,vitalsigns_HeartRate,vitalsigns_RespiratoryRate,...,Lab_RDW-SD,Lab_RDW,Lab_Red Blood Cells,Lab_Sodium,Lab_Urea Nitrogen,Lab_White Blood Cells,Lab_RBC,Lab_Specific Gravity,Lab_pH,Emergency department boarding time
count,29197.000000,22541.000000,24511.000000,22198.000000,23510.000000,24477.000000,24345.000000,26071.000000,27959.000000,27862.000000,...,17193.000000,27684.000000,27695.000000,26461.000000,27715.000000,27714.000000,11677.000000,15473.000000,16387.000000,29197.000000
mean,65.143063,98.117588,91.206091,18.004505,97.333003,129.285100,72.928240,98.257669,87.119622,19.062199,...,50.276700,15.142505,3.901045,137.191829,29.088147,11.841888,10.290486,1.018390,6.232227,71.147636
std,17.409553,1.014572,22.303735,2.417775,2.477151,28.151516,17.416372,0.780968,19.948020,3.993018,...,9.047972,2.525025,0.873870,6.084130,24.429036,11.253058,23.524285,0.009522,0.712160,60.104707
min,21.000000,94.600000,14.000000,10.000000,88.000000,40.000000,4.000000,95.400000,1.000000,4.000000,...,30.600000,0.000000,0.000000,90.000000,1.000000,0.000000,0.000000,1.000000,5.000000,0.350000
25%,54.000000,97.500000,75.000000,16.000000,96.000000,109.000000,61.000000,97.800000,73.000000,16.000000,...,44.000000,13.300000,3.320000,134.000000,14.000000,7.200000,1.000000,1.012000,6.000000,2.966667
50%,67.000000,98.000000,89.000000,18.000000,98.000000,127.000000,72.000000,98.100000,86.000000,18.000000,...,48.300000,14.500000,3.960000,138.000000,21.000000,10.000000,2.000000,1.017000,6.000000,70.000000
75%,79.000000,98.600000,106.000000,20.000000,99.000000,147.000000,84.000000,98.600000,100.000000,21.000000,...,54.500000,16.300000,4.510000,141.000000,35.000000,14.000000,7.000000,1.023000,6.500000,103.000000
max,91.000000,101.600000,177.000000,24.000000,100.000000,245.000000,147.000000,100.900000,169.000000,32.000000,...,126.500000,34.100000,7.750000,180.000000,260.000000,402.500000,182.000000,1.052000,9.000000,605.000000


In [16]:
final_data.iloc[:,:-1]

,Age,Insurance,Race,Arrival transport,triage_Temperature,triage_HeartRate,triage_RespiratoryRate,triage_SpO2,triage_SBP,triage_DBP,...,Diagnosis_R42,Diagnosis_R45,Diagnosis_R50,Diagnosis_R51,Diagnosis_R53,Diagnosis_S72,Diagnosis_X58,Diagnosis_Y83,Diagnosis_Z79,Emergency department boarding time
0,52.0,1,1,1,98.7,77.0,16.0,98.0,96.0,50.0,...,0,0,0,0,0,0,0,0,0,85.0
1,76.0,1,2,2,97.8,80.0,NaN,97.0,132.0,83.0,...,0,0,0,0,0,0,0,0,1,64.0
2,55.0,0,1,2,99.0,81.0,16.0,97.0,160.0,102.0,...,0,0,1,0,1,0,0,0,0,148.0
3,77.0,1,2,2,99.0,80.0,24.0,98.0,116.0,66.0,...,0,0,0,0,0,0,0,0,0,94.0
4,77.0,1,1,1,97.9,64.0,15.0,98.0,130.0,62.0,...,0,0,0,0,0,0,0,0,0,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29192,72.0,0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,62.0
29193,63.0,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,76.0
29194,77.0,1,2,1,98.4,106.0,20.0,88.0,129.0,67.0,...,0,0,0,0,0,0,0,0,0,184.0
29195,77.0,1,2,1,98.9,108.0,20.0,NaN,125.0,54.0,...,0,0,0,0,0,0,0,0,0,2.1
